In [ ]:
# Install required packages (run once)
!pip install mediapipe opencv-python pynput


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
from IPython.display import display, Image, clear_output
from pynput.keyboard import Controller


In [ ]:
class GestureConfig:
    HAND_DETECTION_CONFIDENCE = 0.7
    HAND_TRACKING_CONFIDENCE = 0.5
    POSE_DETECTION_CONFIDENCE = 0.7
    POSE_TRACKING_CONFIDENCE = 0.5

    SHOOTING_WRIST_THRESHOLD = 0.6
    GRENADE_X_THRESHOLD = 0.7
    GRENADE_Y_THRESHOLD = 0.5
    RUNNING_X_THRESHOLD = 0.3
    JUMPING_Y_THRESHOLD = 0.3

class KeyBindings:
    ACTION_KEYS = {
        "idle": None,
        "run": 'd',
        "shoot": 'j',
        "grenade": 'l',
        "jump": 'k'
    }


In [ ]:
class GestureDetector:
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5
        )
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            static_image_mode=False,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5
        )
        self.mp_drawing = mp.solutions.drawing_utils

    def detect_gesture(self, frame):
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        hand_results = self.hands.process(rgb_frame)
        pose_results = self.pose.process(rgb_frame)
        annotated_frame = frame.copy()

        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    annotated_frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)

        if pose_results.pose_landmarks:
            self.mp_drawing.draw_landmarks(
                annotated_frame, pose_results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS)

        action = self._analyze_gesture(hand_results, pose_results, frame.shape)
        cv2.putText(annotated_frame, f"Action: {action}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        return action, annotated_frame

    def _analyze_gesture(self, hand_results, pose_results, frame_shape):
        h, w = frame_shape[:2]
        action = "idle"
        if hand_results.multi_hand_landmarks:
            for landmarks in hand_results.multi_hand_landmarks:
                lm = landmarks.landmark
                wrist_y = lm[0].y * h
                index_y = lm[8].y * h
                middle_y = lm[12].y * h
                wrist_x = lm[0].x * w

                if wrist_y < h * GestureConfig.SHOOTING_WRIST_THRESHOLD and index_y < wrist_y and abs(index_y - middle_y) > 20:
                    action = "shoot"
                elif wrist_x > w * GestureConfig.GRENADE_X_THRESHOLD and wrist_y < h * GestureConfig.GRENADE_Y_THRESHOLD:
                    action = "grenade"
                elif wrist_x < w * GestureConfig.RUNNING_X_THRESHOLD:
                    action = "run"
        if pose_results.pose_landmarks:
            left_shoulder = pose_results.pose_landmarks.landmark[11]
            right_shoulder = pose_results.pose_landmarks.landmark[12]
            shoulder_avg_y = (left_shoulder.y + right_shoulder.y) / 2 * h
            if shoulder_avg_y < h * GestureConfig.JUMPING_Y_THRESHOLD:
                action = "jump"
        return action


In [ ]:
class KeyboardController:
    def __init__(self):
        self.keyboard = Controller()
        self.current_action = "idle"
        self.key_pressed = None
        self.last_action_time = time.time()

    def execute_action(self, action):
        current_time = time.time()
        if current_time - self.last_action_time < 0.1:
            return
        if action != self.current_action:
            self._release_current_key()
            self._press_action_key(action)
            self.current_action = action
            self.last_action_time = current_time

    def _press_action_key(self, action):
        key_map = KeyBindings.ACTION_KEYS
        if action in key_map and key_map[action]:
            try:
                self.keyboard.press(key_map[action])
                self.key_pressed = key_map[action]
                print(f"Pressed: {key_map[action]} for action: {action}")
            except Exception as e:
                print(f"Error pressing key {key_map[action]}: {e}")

    def _release_current_key(self):
        if self.key_pressed:
            try:
                self.keyboard.release(self.key_pressed)
                print(f"Released: {self.key_pressed}")
                self.key_pressed = None
            except Exception as e:
                print(f"Error releasing key: {e}")

    def release_all_keys(self):
        self._release_current_key()
        self.current_action = "idle"


In [ ]:
gesture_detector = GestureDetector()
keyboard_controller = KeyboardController()

cap = cv2.VideoCapture(0)
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        action, annotated_frame = gesture_detector.detect_gesture(frame)
        keyboard_controller.execute_action(action)
        cv2.imshow("Gesture Control", annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    keyboard_controller.release_all_keys()
    cv2.destroyAllWindows()
